# MongoDB - Lab

## Introduction

In this lesson, we'll get some hands-on experience with MongoDB!

## Objectives
You will be able to: 

- Create a MongoDB database   
- Insert data into a MongoDB database   
- Read data from a MongoDB database   
- Update data in a MongoDB database   

## Getting Started

To begin this lab, make sure that you start up the mongoDB server in your terminal first! **You must do this lab locally on your computer, not in Learn.**


## Connecting to the MongoDB Database

In the cell below, import the appropriate library and connect to the mongoDB server. Create a new database called `'lab_db'`.

In [1]:
import pymongo
myclient = pymongo.MongoClient()
myclient.list_database_names()

['admin', 'config', 'example_database', 'lab_db', 'local', 'test']

## Creating a Collection

Now, create a collection called `'lab_collection'` inside `'lab_db'`.

In [2]:
mydb = myclient['lab_db']
mydb

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'lab_db')

In [3]:
mydb.list_collection_names()

['lab_collection']

In [4]:
mycollection = mydb['lab_collection']
mycollection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'lab_db'), 'lab_collection')

## Adding Some Data

Now, we're going to add some example records into our database. In the cells below, create dictionary representations of the following customer records:


|     Name     |            Email           |  Mailing_Address  | Balance |                         Notes                         |
|:------------:|:--------------------------:|:-----------------:|:-------:|:-----------------------------------------------------:|
|  John Smith  |    j.smith@thesmiths.com   | 123 mulberry lane |   0.0   |    Called technical support, issue not yet resolved   |
|  Jane Smith  |  jane_smith@thesmiths.com  |         Null          |  25.00  |                   Null                                    |
|  Adam Enbar  | adam@theflatironschool.com |    11 Broadway    |  14.99  |           Set up on recurring billing cycle           |
| Avi Flombaum |  avi@theflatironschool.com |    11 Broadway    |   0.0   |                   Null                                    |
|   Steven S.  |     steven.s@gmail.net     |         Null          |  -20.23 | Refunded for overpayment due to price match guarantee |


Your first challenge is to take all of the data in the table above and create the corresponding documents and add then to our mongo database. Note that fields that contain 'Null' should not be included in the document (recall that since mongo is schema-less, each document can be different). 

Create the documents from the table listed above, and then use `insert_many()` to insert them into the collection.

In [5]:
import pandas as pd

df = pd.read_clipboard(sep='|')
df

,Unnamed: 0,Name,Email,Mailing_Address,Balance,Notes,Unnamed: 6
0,NaN,:------------:,:--------------------------:,:-----------------:,:-------:,:---------------------------------------------...,NaN
1,NaN,John Smith,j.smith@thesmiths.com,123 mulberry lane,0.0,"Called technical support, issue not yet re...",NaN
2,NaN,Jane Smith,jane_smith@thesmiths.com,Null,25.00,Null ...,NaN
3,NaN,Adam Enbar,adam@theflatironschool.com,11 Broadway,14.99,Set up on recurring billing cycle ...,NaN
4,NaN,Avi Flombaum,avi@theflatironschool.com,11 Broadway,0.0,Null ...,NaN
5,NaN,Steven S.,steven.s@gmail.net,Null,-20.23,Refunded for overpayment due to price match g...,NaN


In [6]:
df.drop(columns=[col for col in df.columns if 'Unnamed' in col],inplace=True)
df.columns

Index(['     Name     ', '            Email           ', '  Mailing_Address  ',
       ' Balance ', '                         Notes                         '],
      dtype='object')

In [7]:
df.columns = [col.strip() for col in  df.columns]
df

,Name,Email,Mailing_Address,Balance,Notes
0,:------------:,:--------------------------:,:-----------------:,:-------:,:---------------------------------------------...
1,John Smith,j.smith@thesmiths.com,123 mulberry lane,0.0,"Called technical support, issue not yet re..."
2,Jane Smith,jane_smith@thesmiths.com,Null,25.00,Null ...
3,Adam Enbar,adam@theflatironschool.com,11 Broadway,14.99,Set up on recurring billing cycle ...
4,Avi Flombaum,avi@theflatironschool.com,11 Broadway,0.0,Null ...
5,Steven S.,steven.s@gmail.net,Null,-20.23,Refunded for overpayment due to price match g...


In [8]:
df.drop(0,inplace=True)
df

,Name,Email,Mailing_Address,Balance,Notes
1,John Smith,j.smith@thesmiths.com,123 mulberry lane,0.0,"Called technical support, issue not yet re..."
2,Jane Smith,jane_smith@thesmiths.com,Null,25.00,Null ...
3,Adam Enbar,adam@theflatironschool.com,11 Broadway,14.99,Set up on recurring billing cycle ...
4,Avi Flombaum,avi@theflatironschool.com,11 Broadway,0.0,Null ...
5,Steven S.,steven.s@gmail.net,Null,-20.23,Refunded for overpayment due to price match g...


In [9]:
# df['Name']
for col in df.columns:
    df[col] = df[col].map(lambda x: x.strip())
df

,Name,Email,Mailing_Address,Balance,Notes
1,John Smith,j.smith@thesmiths.com,123 mulberry lane,0.0,"Called technical support, issue not yet resolved"
2,Jane Smith,jane_smith@thesmiths.com,Null,25.00,Null
3,Adam Enbar,adam@theflatironschool.com,11 Broadway,14.99,Set up on recurring billing cycle
4,Avi Flombaum,avi@theflatironschool.com,11 Broadway,0.0,Null
5,Steven S.,steven.s@gmail.net,Null,-20.23,Refunded for overpayment due to price match gu...


In [10]:
df['Balance'] = df['Balance'].astype(float)
df.dtypes

Name                object
Email               object
Mailing_Address     object
Balance            float64
Notes               object
dtype: object

In [11]:
df['Name'].iloc[0]

'John Smith'

In [12]:
records = df.to_dict('records')
records

[{'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '123 mulberry lane',
  'Balance': 0.0,
  'Notes': 'Called technical support, issue not yet resolved'},
 {'Name': 'Jane Smith',
  'Email': 'jane_smith@thesmiths.com',
  'Mailing_Address': 'Null',
  'Balance': 25.0,
  'Notes': 'Null'},
 {'Name': 'Adam Enbar',
  'Email': 'adam@theflatironschool.com',
  'Mailing_Address': '11 Broadway',
  'Balance': 14.99,
  'Notes': 'Set up on recurring billing cycle'},
 {'Name': 'Avi Flombaum',
  'Email': 'avi@theflatironschool.com',
  'Mailing_Address': '11 Broadway',
  'Balance': 0.0,
  'Notes': 'Null'},
 {'Name': 'Steven S.',
  'Email': 'steven.s@gmail.net',
  'Mailing_Address': 'Null',
  'Balance': -20.23,
  'Notes': 'Refunded for overpayment due to price match guarantee'}]

In [13]:
pd.DataFrame.from_records(records)

,Name,Email,Mailing_Address,Balance,Notes
0,John Smith,j.smith@thesmiths.com,123 mulberry lane,0.00,"Called technical support, issue not yet resolved"
1,Jane Smith,jane_smith@thesmiths.com,Null,25.00,Null
2,Adam Enbar,adam@theflatironschool.com,11 Broadway,14.99,Set up on recurring billing cycle
3,Avi Flombaum,avi@theflatironschool.com,11 Broadway,0.00,Null
4,Steven S.,steven.s@gmail.net,Null,-20.23,Refunded for overpayment due to price match gu...


In [14]:
# customer_1 = None
# customer_2 = None
# customer_3 = None
# customer_4 = None
# customer_5 = None

all_records = records

insertion_results = mycollection.insert_many(all_records)
insertion_results.inserted_ids

[ObjectId('5ebc87a3b25f6cbe7ff9ebf5'),
 ObjectId('5ebc87a3b25f6cbe7ff9ebf6'),
 ObjectId('5ebc87a3b25f6cbe7ff9ebf7'),
 ObjectId('5ebc87a3b25f6cbe7ff9ebf8'),
 ObjectId('5ebc87a3b25f6cbe7ff9ebf9')]

Now, access the appropriate attribute from the results object returned from the insertion to see the unique IDs for each record inserted, so that we can confirm each were inserted correctly. 

## Querying and Filtering

In the cell below, return the name and email address for every customer record. Then, print every item from the query to show that it worked correctly. 

In [15]:
query_1 = mycollection.find({},{'Name': 1,'Email':1})
query_1

In [16]:
list(query_1)

[{'_id': ObjectId('5ebc5d8e6457b8a3fe3c83fd'),
  'Name': 'Jane Smith',
  'Email': 'jane_smith@thesmiths.com'},
 {'_id': ObjectId('5ebc5d8e6457b8a3fe3c83fe'),
  'Name': 'Adam Enbar',
  'Email': 'adam@theflatironschool.com'},
 {'_id': ObjectId('5ebc5d8e6457b8a3fe3c83ff'),
  'Name': 'Avi Flombaum',
  'Email': 'avi@theflatironschool.com'},
 {'_id': ObjectId('5ebc5d8e6457b8a3fe3c8400'),
  'Name': 'Steven S.',
  'Email': 'steven.s@gmail.net'},
 {'_id': ObjectId('5ebc842bbff37c3979cae6e8'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com'},
 {'_id': ObjectId('5ebc842bbff37c3979cae6e9'),
  'Name': 'Jane Smith',
  'Email': 'jane_smith@thesmiths.com'},
 {'_id': ObjectId('5ebc842bbff37c3979cae6ea'),
  'Name': 'Adam Enbar',
  'Email': 'adam@theflatironschool.com'},
 {'_id': ObjectId('5ebc842bbff37c3979cae6eb'),
  'Name': 'Avi Flombaum',
  'Email': 'avi@theflatironschool.com'},
 {'_id': ObjectId('5ebc842bbff37c3979cae6ec'),
  'Name': 'Steven S.',
  'Email': 'steven.s@gmail.net'},
 {'_id':

Great! Now, let's write a query that gets an individual record based on a stored key-value pair a document contains. 

In the cell below, write a query to get the record for `'John Smith'` by using his name. Then, print the results of the query to demonstrate that it worked correctly.  

In [17]:
query_2 = mycollection.find({'Name':'John Smith'})
list(query_2)


[{'_id': ObjectId('5ebc842bbff37c3979cae6e8'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '123 mulberry lane',
  'Balance': '0.0',
  'Notes': 'Called technical support, issue not yet resolved',
  'Mailing Address': '367 55th St., apt 2A'},
 {'_id': ObjectId('5ebc86c0bff37c3979cae6ed'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '123 mulberry lane',
  'Balance': 0.0,
  'Notes': 'Called technical support, issue not yet resolved',
  'Mailing Address': '367 55th St., apt 2A'},
 {'_id': ObjectId('5ebc86f0bff37c3979cae6f3'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '123 mulberry lane',
  'Balance': 0.0,
  'Notes': 'Called technical support, issue not yet resolved',
  'Mailing Address': '367 55th St., apt 2A'},
 {'_id': ObjectId('5ebc87a3b25f6cbe7ff9ebf5'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '123 mulberry lane',
  'Balance': 0.0,
  '

In [18]:
from bson import ObjectId
deleted = mycollection.delete_one({'_id':ObjectId('5ebc5d8e6457b8a3fe3c83fc')})
deleted.deleted_count


0

In [19]:
query_2 = mycollection.find({'Name':'John Smith'})
res2 = list(query_2)


In [20]:
res2[0]['_id']

ObjectId('5ebc842bbff37c3979cae6e8')

Great! Now, write a query to get the names, email addresses, and balances for customers that have a balance greater than 0. Use a modifier to do this. 

**_HINT_**: In the query below, you'll be passing in two separate dictionaries. The first one defines the logic of the query, while the second tells which fields we want returned. 

In [21]:
query_3 = mycollection.find({'Balance':{'$gt':0}},{'Name':1,'Email':1,'Balance':1})
list(query_3)


[{'_id': ObjectId('5ebc5d8e6457b8a3fe3c83fd'),
  'Name': 'Jane Smith',
  'Email': 'jane_smith@thesmiths.com',
  'Balance': 25.0},
 {'_id': ObjectId('5ebc5d8e6457b8a3fe3c83fe'),
  'Name': 'Adam Enbar',
  'Email': 'adam@theflatironschool.com',
  'Balance': 14.99},
 {'_id': ObjectId('5ebc86c0bff37c3979cae6ee'),
  'Name': 'Jane Smith',
  'Email': 'jane_smith@thesmiths.com',
  'Balance': 25.0},
 {'_id': ObjectId('5ebc86c0bff37c3979cae6ef'),
  'Name': 'Adam Enbar',
  'Email': 'adam@theflatironschool.com',
  'Balance': 14.99},
 {'_id': ObjectId('5ebc86f0bff37c3979cae6f4'),
  'Name': 'Jane Smith',
  'Email': 'jane_smith@thesmiths.com',
  'Balance': 25.0},
 {'_id': ObjectId('5ebc86f0bff37c3979cae6f5'),
  'Name': 'Adam Enbar',
  'Email': 'adam@theflatironschool.com',
  'Balance': 14.99},
 {'_id': ObjectId('5ebc87a3b25f6cbe7ff9ebf6'),
  'Name': 'Jane Smith',
  'Email': 'jane_smith@thesmiths.com',
  'Balance': 25.0},
 {'_id': ObjectId('5ebc87a3b25f6cbe7ff9ebf7'),
  'Name': 'Adam Enbar',
  'Email':

## Updating a Record

Now, let's update some records. In the cell below. set the mailing address for `'John Smith'` to `'367 55th St., apt 2A'`.

In [22]:
record_to_update_1 = {'Name':'John Smith'}
update_1 = {'$set':{'Mailing_Address':'367 55th St., apt 2A'}}
mycollection.update_many(record_to_update_1,update_1)

Now, write a query to check that the update worked for this document in the cell below:  

In [23]:
query_4 = mycollection.find(record_to_update_1)
list(query_4)

[{'_id': ObjectId('5ebc842bbff37c3979cae6e8'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '367 55th St., apt 2A',
  'Balance': '0.0',
  'Notes': 'Called technical support, issue not yet resolved',
  'Mailing Address': '367 55th St., apt 2A'},
 {'_id': ObjectId('5ebc86c0bff37c3979cae6ed'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '367 55th St., apt 2A',
  'Balance': 0.0,
  'Notes': 'Called technical support, issue not yet resolved',
  'Mailing Address': '367 55th St., apt 2A'},
 {'_id': ObjectId('5ebc86f0bff37c3979cae6f3'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '367 55th St., apt 2A',
  'Balance': 0.0,
  'Notes': 'Called technical support, issue not yet resolved',
  'Mailing Address': '367 55th St., apt 2A'},
 {'_id': ObjectId('5ebc87a3b25f6cbe7ff9ebf5'),
  'Name': 'John Smith',
  'Email': 'j.smith@thesmiths.com',
  'Mailing_Address': '367 55th St., apt 2A',
  'Balanc

Now, let's assume that we want to add birthdays for each customer record. Consider the following table:

|     Name     |  Birthday  |
|:------------:|:----------:|
|  John Smith  | 02/20/1986 |
|  Jane Smith  | 07/07/1983 |
|  Adam Enbar  | 12/02/1982 |
| Avi Flombaum | 04/17/1983 |
|   Steven S.  | 08/30/1991 |

We want to add birthdays for each person, but we want to do so in a way where we don't have to do the same repetitive task over and over again. This seems like a good opportunity to write a function to handle some of the logic for us!

In the cell below:

* Store the names in the `names_list` variable as a list.
* Store the birthdays in the `birthdays_list` variable as a list.
* Write a function that takes in the two different lists, and updates each record by adding in the appropriate key-value pair containing that user's birthday.

**_Hint:_** There are several ways that you could write this, depending on whether you want to use the `update_one()` method or the `update_many()` method. See if you can figure out both approaches!

In [27]:
names_list =  ['John Smith','Jane Smith','Adam Enbar','Avi Flombaum']
birthdays_list= ['02/20/1986','07/07/1983','12/02/1982','04/17/1983']
data = list(zip(names_list,birthdays_list))
for d in data:
    display(d)

('John Smith', '02/20/1986')

('Jane Smith', '07/07/1983')

('Adam Enbar', '12/02/1982')

('Avi Flombaum', '04/17/1983')

In [28]:
def update_birthdays(names, birthdays):
    data = list(zip(names,birthdays))
    for (name,birthday) in data:
        query = {"Name":name}
        update  = {'$set':{'Birthday':birthday}}
        mycollection.update_many(query,update)
        
update_birthdays(names_list, birthdays_list)

In [33]:
data = dict(zip(names_list,birthdays_list))
for name,birthday in data.items():
    print(name,birthday)

John Smith 02/20/1986
Jane Smith 07/07/1983
Adam Enbar 12/02/1982
Avi Flombaum 04/17/1983


Now, write a query to check your work and see that the birthdays were added correctly.

In [30]:
list(mycollection.find({}))

[{'_id': ObjectId('5ebc5d8e6457b8a3fe3c83fd'),
  'Name': 'Jane Smith',
  'Email': 'jane_smith@thesmiths.com',
  'Mailing_Address': 'Null',
  'Balance': 25.0,
  'Notes': 'Null',
  'Birthday': '07/07/1983'},
 {'_id': ObjectId('5ebc5d8e6457b8a3fe3c83fe'),
  'Name': 'Adam Enbar',
  'Email': 'adam@theflatironschool.com',
  'Mailing_Address': '11 Broadway',
  'Balance': 14.99,
  'Notes': 'Set up on recurring billing cycle',
  'Birthday': '12/02/1982'},
 {'_id': ObjectId('5ebc5d8e6457b8a3fe3c83ff'),
  'Name': 'Avi Flombaum',
  'Email': 'avi@theflatironschool.com',
  'Mailing_Address': '11 Broadway',
  'Balance': 0.0,
  'Notes': 'Null',
  'Birthday': '04/17/1983'},
 {'_id': ObjectId('5ebc5d8e6457b8a3fe3c8400'),
  'Name': 'Steven S.',
  'Email': 'steven.s@gmail.net',
  'Mailing_Address': 'Null',
  'Balance': -20.23,
  'Notes': 'Refunded for overpayment due to price match guarantee',
  'Birthday': '08/30/1991'},
 {'_id': ObjectId('5ebc842bbff37c3979cae6e8'),
  'Name': 'John Smith',
  'Email': 'j

# Using GitHub Links for Content

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-nosql-document-stores-v2-1-onl01-dtsc-pt-041320/master/images/mongodb.jpg" width=200%>


In [34]:
pd.read_csv('https://raw.githubusercontent.com/jirvingphd/fsds_pt_100719_cohort_notes/master/datasets/baltimore_crime_2020_ts.csv')

,Unnamed: 0,burglary,common assault,larceny,robbery - street,auto theft,agg. assault,larceny from auto,shooting,robbery - residence,robbery - commercial,arson,homicide,robbery - carjacking,rape
0,2014-01-01,3.0,19,34,8,9,22,15,2,1,2,NaN,2.0,NaN,4.0
1,2014-01-02,NaN,23,26,4,7,6,10,0,1,1,NaN,3.0,1.0,0.0
2,2014-01-03,NaN,17,16,2,3,11,6,1,1,1,NaN,1.0,0.0,0.0
3,2014-01-04,NaN,23,23,7,14,14,15,0,1,1,NaN,0.0,0.0,0.0
4,2014-01-05,NaN,22,19,10,11,9,13,2,1,0,NaN,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,2020-04-21,NaN,6,8,5,7,11,8,3,0,1,2.0,1.0,0.0,NaN
2303,2020-04-22,NaN,15,12,5,6,15,5,5,0,2,0.0,1.0,1.0,NaN
2304,2020-04-23,NaN,8,13,6,4,6,7,0,2,4,0.0,1.0,NaN,NaN
2305,2020-04-24,NaN,13,11,6,10,6,9,1,0,0,1.0,NaN,NaN,NaN


Great! It looks like the birthdays have been successfully added to every record correctly!

## Summary

In this lesson, we got some hands-on practice working with MongoDB!